This page is taking over the functionality of ongoing_workspace.ipynb, so that I can import the data frame as it is currently saved, and run things more efficiently, now that Tasks 1-4 are complete. 

In [30]:
# Preamble - imports 
import numpy as np
import pandas as pd  
import data_cleaning_for_EDA as dc 
import matplotlib.pyplot as plt
import seaborn as sns
import imputing_methods as im 

In [31]:
# Preamble - Multiple Inheritance Class 
class MultipleInheritanceTestClass(im.Plotter, im.DataFrameTransform, dc.DataFrameInfo):
    def export_to_csv(self, filename):
        self.data_frame.to_csv(filename, index=False)
        print(f"DataFrame exported to {filename}")

In [17]:
# Preamble - data read in
df_df = pd.read_csv("df_end_of_Task_4.csv")
# ensure it's a data frame at this stage so everything inherits correctly in the next step. 
type(df_df)


pandas.core.frame.DataFrame

In [32]:
df = MultipleInheritanceTestClass(df_df)

Task 5: Remove outliers  (Let the record show that as a statistician I do not condone willfully ignoring extereme-yet-possible data! I do so for the purposes of this task.)

Step 1: Visualise with Plotter class to determine if coumns contain outliers. 
I think the best way to indicate that something is an outlier is to plot the actual values against the expected values (were the data Gaussian) in a QQNorm plot. However, since I already plotted loads of this data and identified many non-Gaussian-looking distributions (that's even after "skew adjustment," another statistically questionable practise), I don't think a Normal QQ plot would tell me anything meaningful about outliers. 

I think the best I can offer right now is a box plot. It turs out there's a handy showfliers argument in the sns.boxplot() method-- see Plotter class, where I've added the boxplot_with_outliers() method. 

In [38]:
# Troubleshooting checks 
print(type(df))

print(type(df.col_names()))

df.stats()



<class '__main__.MultipleInheritanceTestClass'>
Index(['Unnamed: 0', 'id', 'member_id', 'loan_amount', 'funded_amount',
       'funded_amount_inv', 'term_length_in_months', 'int_rate', 'instalment',
       'grade', 'sub_grade', 'years_of_employment', 'home_ownership',
       'annual_inc', 'verification_status', 'issue_date', 'loan_status',
       'payment_plan', 'purpose', 'dti', 'delinq_2yrs', 'earliest_credit_line',
       'inq_last_6mths', 'open_accounts', 'total_accounts', 'out_prncp',
       'out_prncp_inv', 'total_payment', 'total_payment_inv',
       'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries',
       'collection_recovery_fee', 'last_payment_date', 'last_payment_amount',
       'next_payment_date', 'last_credit_pull_date',
       'collections_12_mths_ex_med', 'policy_code', 'application_type'],
      dtype='object')
<class 'NoneType'>
         Unnamed: 0            id     member_id   loan_amount  funded_amount  \
count  54231.000000  5.423100e+04  5.42

In [37]:
column_names = df.col_names()

for column in column_names:
    print(f"Plotting boxplot for {column}:")
    df.boxplot_with_outliers(column) 
    

Index(['Unnamed: 0', 'id', 'member_id', 'loan_amount', 'funded_amount',
       'funded_amount_inv', 'term_length_in_months', 'int_rate', 'instalment',
       'grade', 'sub_grade', 'years_of_employment', 'home_ownership',
       'annual_inc', 'verification_status', 'issue_date', 'loan_status',
       'payment_plan', 'purpose', 'dti', 'delinq_2yrs', 'earliest_credit_line',
       'inq_last_6mths', 'open_accounts', 'total_accounts', 'out_prncp',
       'out_prncp_inv', 'total_payment', 'total_payment_inv',
       'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries',
       'collection_recovery_fee', 'last_payment_date', 'last_payment_amount',
       'next_payment_date', 'last_credit_pull_date',
       'collections_12_mths_ex_med', 'policy_code', 'application_type'],
      dtype='object')


TypeError: 'NoneType' object is not iterable